# 현재 해결해야 할 것
## 1. zolgima-control에 새 start-stream이 안들어오면 잠시 대기
## 2. 여러 session이 입력으로 들어올 경우 처리

In [1]:
import json
import signal
import sys
import datetime
import math
import time
import pandas as pd
import numpy as np
from collections import Counter, deque
import tensorflow as tf
from confluent_kafka import Producer, Consumer, KafkaException, KafkaError

In [2]:
FEATURE_COLS = [
    "EAR",
    "MAR",
    "yawn_rate_per_min",
    "blink_rate_per_min",
    "avg_blink_dur_sec",
    "longest_eye_closure_sec",
]

frame_window_size = 150   # 프레임 단위 윈도우(10단위 반올림)
hop_frame_size = 30       # 프레임 단위 홉(10단위 반올림)

kafka_conf = {
    "bootstrap.servers": "kafka.dongango.com:9094",
    "group.id": "drowsy-session_consumer",
    "enable.auto.commit": False,  # 수동 커밋
    "auto.offset.reset": "earliest",
    "session.timeout.ms": 45000
}

KAFKA_TOPIC = "zolgima-control"

window = deque(maxlen=frame_window_size)
frame_numbers = deque(maxlen=frame_window_size)
CLS_LEVELS = [-1, 0, 1, 2, 3]

model = tf.keras.models.load_model("model.keras")

In [3]:
def fit_standardizer(X: np.ndarray):
    """
    채널별 표준화 스케일러(평균/표준편차) 반환.
    """
    # X shape: (N, T, C)
    mean = X.reshape(-1, X.shape[-1]).mean(axis=0)
    std = X.reshape(-1, X.shape[-1]).std(axis=0)
    std = np.where(std < 1e-8, 1.0, std)
    return mean.astype(np.float32), std.astype(np.float32)

def apply_standardizer(X: np.ndarray, mean: np.ndarray, std: np.ndarray):
    return (X - mean) / std

In [ ]:
def preprocess_value(value_dict):
    """필요한 feature만 뽑아 float 배열로 변환"""
    return [float(value_dict.get(col, np.nan)) for col in FEATURE_COLS]

def predict_and_publish(input, frame_no, model, topic):
       X = np.array(input).reshape(1, frame_window_size, len(FEATURE_COLS))
        
       # ✅ 표준화 적용
       mean, std = fit_standardizer(X)
       X = apply_standardizer(X, mean, std)

       y_pred = model.predict(X, verbose=0)
       class_index = int(np.argmax(y_pred, axis=1)[0])   # multi-class softmax 결과
       drowsy_level = CLS_LEVELS[class_index]
        
       result = {
              "frame": int(frame_no[-1]),  # 마지막 frame 번호
              "drowsy-level": drowsy_level
       }

       producer = Producer({"bootstrap.servers": kafka_conf["bootstrap.servers"]})
        
       producer.produce(
              topic,
              key="drowny-lstm-result",
              value=json.dumps(result)
       )
       producer.flush()
       print("Published:", result)

In [43]:
running = True
all_streams = []
started_streams = [0]
stopped_streams = []
last_started_stream = None

def handle_sigint(signum, frame):
    global running
    running = False

c = Consumer(kafka_conf)
c.subscribe([KAFKA_TOPIC])
IDLE_TIMEOUT_SEC = 5

signal.signal(signal.SIGINT, handle_sigint)
print(f"👂 Subscribed to {KAFKA_TOPIC}. Press Ctrl+C to stop.")

last_msg_ts = time.monotonic()
try:
       while running:
              msg = c.poll(1.0)  # 1초 대기
              if msg is None:
                     if time.monotonic() - last_msg_ts >= IDLE_TIMEOUT_SEC:
                       print("🛑 Idle timeout reached. Stopping session consumer...")
                       break
                     continue
              if msg.error():
                # 파티션 EOF 등 무해한 에러 처리
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    continue
                raise KafkaException(msg.error())

            # 정상 메시지 처리
              try:
                     value = msg.value()
                     data = json.loads(value.decode("utf-8")) if value else None
                     key = msg.key().decode("utf-8") if msg.key() else None
                     print(f"[{msg.topic()} p{msg.partition()} @ {msg.offset()}] "
                     f"key={key} data={data}")
                     if key == "start-stream":
                        started_streams.append(data['session-id'])
                     elif key == "stop-stream":
                        stopped_streams.append(data['session-id'])
                     last_started_stream = started_streams[-1]
              except json.JSONDecodeError as e:
                     print(f"⚠ JSON decode error: {e}; raw={msg.value()!r}")

              if started_streams[-1] == last_started_stream:
                      time.sleep(2)
                      continue

              
finally:
       print("🛑 Closing consumer...")
       c.close()
last_started_stream = started_streams[-1] if started_streams else None

👂 Subscribed to zolgima-control. Press Ctrl+C to stop.
[zolgima-control p0 @ 0] key=start-stream data={'sesstion-id': 'sess-5t1toq4k-mes66sgo', 'session-id': 'sess-5t1toq4k-mes66sgo', 'type': 'jpeg'}
[zolgima-control p0 @ 1] key=stop-stream data={'sesstion-id': 'sess-5t1toq4k-mes66sgo', 'session-id': 'sess-5t1toq4k-mes66sgo', 'type': 'jpeg'}
[zolgima-control p0 @ 2] key=start-stream data={'session-id': 'sess-7ycx8mo2-mes8s5mb', 'type': 'jpeg'}
[zolgima-control p0 @ 3] key=stop-stream data={'session-id': 'sess-7ycx8mo2-mes8s5mb', 'type': 'jpeg'}
[zolgima-control p0 @ 4] key=start-stream data={'session-id': 'sess-zwrb7967-mes8vwu6', 'type': 'jpeg'}
[zolgima-control p0 @ 5] key=start-stream data={'session-id': 'sess-b3f64s77-mes9356t', 'type': 'jpeg'}
🛑 Closing consumer...


In [ ]:
# --------------------------------------------------------------------------
window = deque(maxlen=frame_window_size)
frame_numbers = deque(maxlen=frame_window_size)

session_conf = {
        **kafka_conf,
        "group.id": "zolgima-session_consumer",
        "enable.auto.commit": False,  # 수동 커밋
        "auto.offset.reset": "earliest",  # 처음부터 읽고 싶을 때
        "session.timeout.ms": 45000,
}
c2 = Consumer(session_conf)

c2.subscribe([started_streams[-1]])
frames_data = []
session_running = True

def handle_sigint_session(signum, frame):
    global session_running
    session_running = False

signal.signal(signal.SIGINT, handle_sigint_session)

last_msg_ts = time.monotonic()
try:
       while session_running:
              msg = c2.poll(1.0)  # 1초 대기
              if msg is None:
              
                continue
              if msg.error():
                # 파티션 EOF 등 무해한 에러 처리
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    continue
                raise KafkaException(msg.error())

            # 정상 메시지 처리
              try:
                     value = msg.value()
                     data = json.loads(value.decode("utf-8")) if value else None
                     key = msg.key().decode("utf-8") if msg.key() else None
                     print(f"[{msg.topic()} p{msg.partition()} @ {msg.offset()}] "
                     f"key={key} data={data}")
                     frames_data.append(data)
                     frame_no = int(data.get("frame"))
                     frame_numbers.append(frame_no)
                     window.append([data['metrics'].get(col, np.nan) for col in FEATURE_COLS])
                     if (msg.offset()+1) % frame_window_size == 0:
                            if time.monotonic() - last_msg_ts >= 1:
                                   frame_df = pd.DataFrame(frames_data)
                                   predict_and_publish(window, frame_numbers, model, started_streams[-1]+"-LSTM")
                                   for _ in range(hop_frame_size):
                                          window.popleft()
                                          frame_numbers.popleft()
                     last_msg_ts = time.monotonic()
              except json.JSONDecodeError as e:
                     print(f"⚠ JSON decode error: {e}; raw={msg.value()!r}")
finally:
       print("🛑 Closing consumer...")
       c2.close()

[sess-3fzhrotz-mev696bb p0 @ 0] key=None data={'topic': 'sess-3fzhrotz-mev696bb', 'frame': 1, 'source_idx': 1, 'timestamp_ms': 1756371636156, 'rel_time_sec': 0.0, 'metrics': {'label_name': 'eyes_state/open', 'EAR': 0.2966610960210734, 'MAR': 0.0022004342718319756, 'yawn_rate_per_min': 0.0, 'blink_rate_per_min': 0.0, 'avg_blink_dur_sec': nan, 'longest_eye_closure_sec': nan, 'ear_left': 0.29837945330365145, 'ear_right': 0.29494273873849536, 'ear_mean': 0.2966610960210734, 'mar': 0.0022004342718319756, 'face_found': True}}
🛑 Closing consumer...


NameError: name 'window_frame_size' is not defined